In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 텍스트-SQL 변환

이 튜토리얼에서는 `smolagents`를 사용하여 SQL을 활용하는 에이전트를 구현하는 방법을 살펴보겠습니다.

> 황금 질문부터 시작해 봅시다: 왜 간단하게 표준 텍스트-SQL 파이프라인을 사용하지 않을까요?

표준 텍스트-SQL 파이프라인은 생성된 SQL 쿼리가 정확하지 않을 수 있으므로 취약합니다. 설상가상으로 쿼리가 정확하지 않지만 오류를 발생시키지 않고 대신 경고를 울리지 않고 부정확하거나 쓸모없는 출력을 제공할 수 있습니다.

👉 대신 에이전트 시스템은 출력을 비판적으로 검사하고 쿼리를 변경해야 하는지 여부를 결정할 수 있으므로 성능이 크게 향상됩니다.

이 에이전트를 만들어 봅시다! 💪

필요한 종속성을 설치하려면 아래 줄을 실행하십시오.
```bash
!pip install smolagents python-dotenv sqlalchemy --upgrade -q
```
추론 공급자를 호출하려면 환경 변수 `HF_TOKEN`으로 유효한 토큰이 필요합니다.
이를 로드하기 위해 python-dotenv를 사용합니다.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

그런 다음 SQL 환경을 설정합니다.

In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

### 에이전트 만들기

이제 SQL 테이블을 도구로 검색할 수 있도록 만들어 보겠습니다.

도구의 설명 속성은 에이전트 시스템에 의해 LLM의 프롬프트에 포함됩니다. 이는 LLM에 도구 사용 방법에 대한 정보를 제공합니다. 여기서 SQL 테이블을 설명하려고 합니다.

In [ ]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "열:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

```text
열:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT
```

이제 도구를 만들어 보겠습니다. 다음이 필요합니다. (자세한 내용은 [도구 문서](https://huggingface.co/docs/smolagents/main/ko/examples/../tutorials/tools)를 참조하십시오.)
- 인수를 나열하는 `Args:` 부분이 있는 독스트링입니다.
- 입력과 출력 모두에 대한 유형 힌트입니다.

In [ ]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    테이블에서 SQL 쿼리를 수행할 수 있습니다. 결과의 문자열 표현을 반환합니다.
    테이블 이름은 'receipts'입니다. 설명은 다음과 같습니다.
        열:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: 수행할 쿼리입니다. 올바른 SQL이어야 합니다.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

이제 이 도구를 활용하는 에이전트를 만들어 보겠습니다.

smolagents의 주요 에이전트 클래스인 `CodeAgent`를 사용합니다. 이 에이전트는 코드에 작업을 작성하고 ReAct 프레임워크에 따라 이전 출력에 대해 반복할 수 있습니다.

모델은 에이전트 시스템을 구동하는 LLM입니다. `InferenceClientModel`을 사용하면 서버리스 또는 전용 엔드포인트를 통해 HF의 추론 API를 사용하여 LLM을 호출할 수 있지만 모든 독점 API를 사용할 수도 있습니다.

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="meta-llama/Llama-3.1-8B-Instruct"),
)
agent.run("가장 비싼 영수증을 받은 고객의 이름을 알려주시겠어요?")

### 레벨 2: 테이블 조인

이제 더 어렵게 만들어 봅시다! 에이전트가 여러 테이블 간의 조인을 처리하도록 하려고 합니다.

따라서 각 receipt_id에 대한 웨이터 이름을 기록하는 두 번째 테이블을 만들어 봅시다!

In [ ]:
table_name = "waiters"
waiters = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
insert_rows_into_table(rows, waiters)

테이블을 변경했으므로 LLM이 이 테이블의 정보를 제대로 활용할 수 있도록 이 테이블의 설명으로 `SQLExecutorTool`을 업데이트합니다.

In [ ]:
updated_description = """테이블에서 SQL 쿼리를 수행할 수 있습니다. 이 도구의 출력은 실행 출력의 문자열 표현이라는 점에 유의하십시오.
다음 테이블을 사용할 수 있습니다."""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"테이블 '{table}':\n"

    table_description += "열:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

이 요청은 이전 요청보다 약간 더 어려우므로 LLM 엔진을 더 강력한 [Qwen/Qwen2.5-Coder-32B-Instruct](https://huggingface.co/Qwen/Qwen2.5-Coder-32B-Instruct)로 전환합니다!

In [ ]:
sql_engine.description = updated_description

agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct"),
)

agent.run("팁으로 가장 많은 돈을 받은 웨이터는 누구인가요?")

바로 작동합니다! 설정이 놀랍도록 간단했죠?

이 예제는 완료되었습니다! 다음 개념을 다루었습니다.
- 새 도구 만들기.
- 도구 설명 업데이트.
- 더 강력한 LLM으로 전환하면 에이전트 추론에 도움이 됩니다.

✅ 이제 항상 꿈꿔 왔던 이 텍스트-SQL 시스템을 만들 수 있습니다! ✨